In [1087]:
import librosa
import tensorflow as tf
import numpy as np
import audioop
import pyaudio
import wave
import speech_recognition as sr
import os

In [1088]:
SAVED_MODEL_PATH = "batch8 175.h5"
SAMPLES_TO_CONSIDER = 22050

In [1089]:
class _Keyword_Spotting_Service:

    model = None
    _mapping = [
        "Kanan",
        "Kiri",
        "Maju",
        "Stop"
    ]
    _instance = None


    def predict(self, file_path):

        # extract MFCC
        MFCCs = self.preprocess(file_path)

        # we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
        MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

        # get the predicted label
        predictions = self.model.predict(MFCCs)
        predicted_index = np.argmax(predictions)
        print(predictions)
        predicted_keyword = self._mapping[predicted_index]  
        return predicted_keyword


    def preprocess(self, file_path, num_mfcc=13, n_fft=2048, hop_length=512):

        # load audio file
        signal, sample_rate = librosa.load(file_path)

        if len(signal) >= SAMPLES_TO_CONSIDER:
            # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]

            # extract MFCCs
            MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        return MFCCs.T

In [1090]:
def Keyword_Spotting_Service():

    # ensure an instance is created only the first time the factory function is called
    if _Keyword_Spotting_Service._instance is None:
        _Keyword_Spotting_Service._instance = _Keyword_Spotting_Service()
        _Keyword_Spotting_Service.model = tf.keras.models.load_model(SAVED_MODEL_PATH)
    return _Keyword_Spotting_Service._instance

In [1091]:
FRAMES = []
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 2250
RECORD_SECONDS = 2
CHUNK = 1024
TEMPORARY_WAVE_FILENAME = "temp.wav"

mic = sr.Microphone()
rec = sr.Recognizer()
audio = pyaudio.PyAudio()

In [1092]:
def recording():
    global FRAMES, FORMAT, CHUNK, CHANNELS, RATE, TEMPORARY_WAVE_FILENAME
    try:
        print ("recording...")
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            mx = audioop.max(data, 2)
            # print mx
            FRAMES.append(data)
        print ("Finish recordings")

        stream.stop_stream()
        stream.close()
        audio.terminate()

        waveFile = wave.open(TEMPORARY_WAVE_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(FRAMES))
        waveFile.close()
        del stream
    except Exception as e:
        print (e.message)
        raise

In [1093]:
def recognise():
    global FRAMES
    kss = Keyword_Spotting_Service()
    r = sr.Recognizer()
    with sr.AudioFile(TEMPORARY_WAVE_FILENAME) as source:
        audio = r.record(source)  # read the entire audio file
    keyword = kss.predict(TEMPORARY_WAVE_FILENAME)
    os.remove(TEMPORARY_WAVE_FILENAME)
    try:
        print("Perintah: " + keyword)
    except sr.UnknownValueError:  # speech is unintelligible
        print("Tidak dapat menerjemahkan")
    
    # create 2 instances of the keyword spotting service
    
    #kss1 = Keyword_Spotting_Service()

    # check that different instances of the keyword spotting service point back to the same object (singleton)
    #assert kss is kss1

    # make a prediction
    #keyword = kss.predict(audio)
    #print(keyword)

In [1094]:
if __name__ == "__main__":
    #while True :
        recording()
        recognise()
        #recording2()

recording...
Finish recordings


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14728\1735644397.py:39: FutureWarning: Pass y=[1.1198784e-06 2.5700620e-06 3.9386314e-06 ... 3.7597943e-02 2.3017019e-02
 9.0650665e-03], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)


1/1 [==============================] - 0s 383ms/step
[[0.00174168 0.08270929 0.80764794 0.10790103]]
Perintah: Maju
